In [17]:
import pandas as pd 
import matplotlib.pyplot as plt 
!pip install pyecharts

In [4]:
data = pd.read_csv('超市运营数据.csv',encoding='GBK',parse_dates=['成交时间'])

In [5]:
data.head()

,商品ID,类别ID,门店编号,单价,销量,成交时间,订单ID
0,30006206,915000003,CDNL,25.23,0.328,2020-10-03 09:56:00,20201003CDLG000210052759
1,30163281,914010000,CDNL,2.00,2.000,2020-10-03 09:56:00,20201003CDLG000210052759
2,30200518,922000000,CDNL,19.62,0.230,2020-10-03 09:56:00,20201003CDLG000210052759
3,29989105,922000000,CDNL,2.80,2.044,2020-10-03 09:56:00,20201003CDLG000210052759
4,30179558,915000100,CDNL,47.41,0.226,2020-10-03 09:56:00,20201003CDLG000210052759


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3478 entries, 0 to 3477
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   商品ID    3478 non-null   int64         
 1   类别ID    3478 non-null   int64         
 2   门店编号    3478 non-null   object        
 3   单价      3478 non-null   float64       
 4   销量      3478 non-null   float64       
 5   成交时间    3478 non-null   datetime64[ns]
 6   订单ID    3478 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 190.3+ KB


In [9]:
data['销售额'] = data['单价']*data['销量']
data.head()

,商品ID,类别ID,门店编号,单价,销量,成交时间,订单ID,销售额
0,30006206,915000003,CDNL,25.23,0.328,2020-10-03 09:56:00,20201003CDLG000210052759,8.27544
1,30163281,914010000,CDNL,2.00,2.000,2020-10-03 09:56:00,20201003CDLG000210052759,4.00000
2,30200518,922000000,CDNL,19.62,0.230,2020-10-03 09:56:00,20201003CDLG000210052759,4.51260
3,29989105,922000000,CDNL,2.80,2.044,2020-10-03 09:56:00,20201003CDLG000210052759,5.72320
4,30179558,915000100,CDNL,47.41,0.226,2020-10-03 09:56:00,20201003CDLG000210052759,10.71466


In [7]:
data_kind_group = data.groupby('类别ID')['销量'].sum().reset_index()
data_kind_group

,类别ID,销量
0,910000000,24.0
1,910010000,7.0
2,910010002,1.0
3,910010101,6.0
4,910010301,2.0
...,...,...
363,936030100,1.0
364,936050100,1.0
365,936080103,1.0
366,938030000,8.0


In [10]:
data_kind_group_top10 = data_kind_group.sort_values(by='销量',ascending=False).head(10)
data_kind_group_top10

,类别ID,销量
240,922000003,425.328
239,922000002,206.424
251,923000006,190.294
216,915030104,175.059
238,922000001,121.355
367,960000000,121.000
234,920090000,111.565
249,923000002,91.847
237,922000000,86.395
247,923000000,85.845


In [12]:
data_product_group_top10 = data.groupby('商品ID')['销量'].sum().reset_index().sort_values(by='销量',ascending=False).head(10)
data_product_group_top10

,商品ID,销量
8,29989059,391.549
18,29989072,102.876
469,30022232,101.000
523,30031960,99.998
57,29989157,72.453
476,30023041,64.416
505,30026255,62.375
7,29989058,56.052
510,30027007,48.757
903,30171264,45.000


In [14]:
data_store_sale = data.groupby('门店编号')['销售额'].sum().reset_index().sort_values(by='销售额',ascending=False)
data_store_sale

,门店编号,销售额
0,CDLG,10908.82612
2,CDXL,9981.76166
1,CDNL,8059.47867


In [24]:
from pyecharts import options as opts
from pyecharts.charts import Pie
#绘制不同门店销售额的饼状图
xlist = list(data_store_sale['门店编号'])
ylist = list(data_store_sale['销售额'])
pie = (
    Pie()
    .add(
        '不同门店的销售额',
        [(i,j) for i,j in zip(xlist,ylist)],
        radius = ['30%','75%'], #内半径和外半径
        center = ['50%','50%'],
        rosetype = 'radius', # 所有部分内角度相等，仅通过半径大小表示比例多少
        label_opts = opts.LabelOpts(is_show=False),
    
    )
    .set_global_opts(title_opts = opts.TitleOpts(title='门店销售占比'))
    .set_series_opts(label_opts = opts.LabelOpts(formatter='{b}: {d}'))
)
pie.render_notebook()

In [34]:
from pyecharts.charts import Bar 
bar = (
    Bar()
        .add_xaxis(list(data_kind_group_top10['类别ID']))
        .add_yaxis('销量',list(data_kind_group_top10['销量'].astype(int)))
        .reversal_axis()
        .set_series_opts(label_opts=opts.LabelOpts(position='right'))
        .set_global_opts(title_opts=opts.TitleOpts(title='销售量排名前十的商品类别'))

)
bar.render_notebook()

In [35]:
from pyecharts.charts import Bar 
bar = (
    Bar()
        .add_xaxis(list(data_product_group_top10['商品ID']))
        .add_yaxis('销量',list(data_product_group_top10['销量'].astype(int)))
        .reversal_axis()
        .set_series_opts(label_opts=opts.LabelOpts(position='right'))
        .set_global_opts(title_opts=opts.TitleOpts(title='销售量排名前十的商品'))

)
bar.render_notebook()

In [36]:
#分析超市客流高峰时间段
data['小时'] = data['成交时间'].map(lambda x:int(x.strftime('%H')))
data 

,商品ID,类别ID,门店编号,单价,销量,成交时间,订单ID,销售额,小时
0,30006206,915000003,CDNL,25.23,0.328,2020-10-03 09:56:00,20201003CDLG000210052759,8.27544,9
1,30163281,914010000,CDNL,2.00,2.000,2020-10-03 09:56:00,20201003CDLG000210052759,4.00000,9
2,30200518,922000000,CDNL,19.62,0.230,2020-10-03 09:56:00,20201003CDLG000210052759,4.51260,9
3,29989105,922000000,CDNL,2.80,2.044,2020-10-03 09:56:00,20201003CDLG000210052759,5.72320,9
4,30179558,915000100,CDNL,47.41,0.226,2020-10-03 09:56:00,20201003CDLG000210052759,10.71466,9
...,...,...,...,...,...,...,...,...,...
3473,30031870,915030401,CDXL,6.58,0.862,2020-10-03 10:59:00,20201003CDLG000510025147,5.67196,10
3474,30008276,911010501,CDXL,15.42,0.481,2020-10-03 11:00:00,20201003CDLG000510025148,7.41702,11
3475,30033854,910000000,CDXL,9.62,1.000,2020-10-03 11:17:00,20201003CDLG000510025149,9.62000,11
3476,30206214,910000000,CDXL,15.62,1.000,2020-10-03 11:40:00,20201003CDLG000510025150,15.62000,11


In [38]:
#按小时和订单去重
order_hour = data[['小时','订单ID']].drop_duplicates()
order_hour

,小时,订单ID
0,9,20201003CDLG000210052759
10,9,20201003CDLG000210052760
15,9,20201003CDLG000210052761
22,10,20201003CDLG000210052762
23,10,20201003CDLG000210052763
...,...,...
3472,10,20201003CDLG000510025147
3474,11,20201003CDLG000510025148
3475,11,20201003CDLG000510025149
3476,11,20201003CDLG000510025150


In [48]:
#计算每小时的订单量
order_hour_count = order_hour.groupby('小时')['订单ID'].count().reset_index()
order_hour_count

,小时,订单ID
0,6,10
1,7,37
2,8,106
3,9,156
4,10,143
5,11,63
6,13,30
7,14,36
8,15,17
9,16,50


In [55]:
#用折线图画
import pyecharts.options as opts
from pyecharts.charts import Line
x=list(str(i) for i in list(range(6,22)))
y=list(order_hour_count['订单ID'])
line = (
    Line()
    .add_xaxis(xaxis_data=x)
    .add_yaxis(series_name='销量',y_axis=y,is_smooth=True)
    .set_global_opts(
        title_opts=opts.TitleOpts(title='不同时段销量折线图'),
        yaxis_opts=opts.AxisOpts(
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
    )
)
line.render_notebook()